In [1]:
import pandas as pd
import numpy as np

In [2]:
symbol_geneid_map = pd.read_pickle('http://115.24.151.181/ID_convert/Gene_Symbol-Gene_ID.pkl')

In [3]:
clinicaltrial = pd.read_csv('./clinicaltrial.csv')
clinicaltrial_all = clinicaltrial['ChEMBL'].unique()

clinicaltrial = pd.read_csv('./clinicaltrial.csv')
clinicaltrial_approv = clinicaltrial[clinicaltrial['Clinical_phase']==4]['ChEMBL'].unique()

In [4]:
drugs = pd.read_csv('./drug_targets_dir.csv')

In [5]:
drugs_chembl = pd.read_csv('./drugs_chembl.txt',sep='\t',header=None)
drugs_chembl = drugs_chembl.drop_duplicates(0)
drugs_chembl = pd.Series(index=drugs_chembl[0].values, data=drugs_chembl[1].values)
drugs['ChEMBL_guess'] = drugs['drug_claim_primary_name'].map(drugs_chembl)
drugs['ChEMBL'] = drugs['ChEMBL_guess'].where(drugs['drug_concept_id'].isnull(),drugs['drug_concept_id'])

In [6]:
drugs = drugs.drop_duplicates(['ChEMBL','gene_name'])

In [54]:
drugs.to_csv('./drug_targets_dir_maxphase.csv',index=False)

In [7]:
drugs = pd.read_csv('./drug_targets_dir_maxphase.csv')
drugs['Clinical_phase'] = drugs['Clinical_phase'].fillna(0).astype(int)
drugs = drugs[drugs['ChEMBL'].notnull()]

In [8]:
rwr_res = pd.read_csv('./rwr_res.csv')
rwr_res = rwr_res[rwr_res['source'].isin(drugs['entrez_id'].values)]
direc = pd.DataFrame(columns=rwr_res.columns)
direc['source'] = drugs['entrez_id'].unique()
direc['target'] = direc['source']
direc['prob'] = 1.0
direc['path'] = ''
direc['direction'] = 1
rwr_res = pd.concat([direc,rwr_res], ignore_index=True)

In [9]:
host = pd.read_excel('../source_with_direction/all_host_genes_direction.xlsx')
host['entrez_host'] = host['Host Gene'].map(symbol_geneid_map)
host = host[host['entrez_host'].notnull()]
host['entrez_host'] = host['entrez_host'].astype(int)

In [10]:
map_network = rwr_res.merge(host,left_on='target',right_on='entrez_host',how='right')
map_network = map_network.rename(columns={'direction':'G2G','Direction':'G2V'})
reposition = drugs.merge(map_network, left_on='entrez_id',right_on='source',how='right')
reposition['D2V'] = reposition['Direction']*reposition['G2G']*reposition['G2V']
failed = reposition[reposition['D2V']==1]
reposition.to_csv('./all_DGVs.txt',sep='\t',index=False)
reposition = reposition[reposition['D2V']==-1]

In [12]:
len(reposition['ChEMBL'].unique())

6243

## side effect and comorbidty

In [13]:
snp2disease = pd.read_csv('./variant_gene_direction.txt')
snp2disease = snp2disease[snp2disease['Finemapping Pro']>=0]
snp2disease['entrez_disease'] = snp2disease['gene_name'].map(symbol_geneid_map)
snp2disease = snp2disease[snp2disease['entrez_disease'].notnull()]
snp2disease['entrez_disease'] = snp2disease['entrez_disease'].astype(int)
snp2disease['G2V'] = snp2disease['DirCE'].map({'I':-1,'A':1})

In [14]:
side_cormo_map = rwr_res.merge(snp2disease,left_on='target',right_on='entrez_disease',how='right')
side_cormo_map = side_cormo_map.rename(columns={'direction':'G2G'})
side_cormo = drugs.merge(side_cormo_map, left_on='entrez_id',right_on='source',how='right')
side_cormo['D2V'] = side_cormo['Direction']*side_cormo['G2G']*side_cormo['G2V']
side_cormo = side_cormo[side_cormo['source']==side_cormo['target']]
side_effect = side_cormo[side_cormo['D2V']==1]
cormobidity = side_cormo[side_cormo['D2V']==-1]

In [17]:
len(cormobidity['ChEMBL'].unique())

269

In [63]:
side_cormo.to_csv('./side_cormo.txt',sep='\t', index=False)

In [28]:
side_effect_map = pd.Series(dtype=str)
side_effect_counts = pd.Series(dtype=int)
for drug, sub_df in side_effect.groupby('drug_claim_primary_name'):
    diseases = sub_df['Disease'].unique()
    side_effect_map[drug] = ';'.join(diseases)
    side_effect_counts[drug] = len(diseases)

cormobidity_map = pd.Series(dtype=str)
cormobidity_counts = pd.Series(dtype=int)
for drug, sub_df in cormobidity.groupby('drug_claim_primary_name'):
    diseases = sub_df['Disease'].unique()
    cormobidity_map[drug] = ';'.join(diseases)
    cormobidity_counts[drug] = len(diseases)

## Annotation

In [29]:
anno_reposition = reposition[['drug_claim_primary_name','Virus','Host Gene','Host Gene Type','Host Gene Source',
                            'G2V','Direciton Type', 'Direction Source','gene_name','interaction_type','interaction_group_score','Direction','path','prob', 'ChEMBL','Clinical_phase']].copy()
anno_reposition.columns = ['Drug','Virus','Host Gene','Host Gene Type','Host Gene Source',
                            'Direction','Direction Type', 'Direction Source','Drug Target','DTI Type','DTI Score','Drug Effect','Path','RWR Prob', 'CHEMBL ID','Clinical Phase']

In [30]:
anno_reposition['fake score'] = anno_reposition['DTI Score'].where(anno_reposition['RWR Prob']<1, 10000)
anno_reposition = anno_reposition.sort_values('fake score', ascending=False)
anno_reposition = anno_reposition.drop_duplicates(['CHEMBL ID', 'Host Gene', 'Drug Target'])

In [31]:
anno_reposition.to_csv('./anno_repostition.txt', sep='\t', index=False)

In [32]:
anno_reposition[
    'Direction Type Score'] = anno_reposition[
        'Direction Type'].map({
            'DEG': 0.5,
            'CRISPR-Screening': 1,
            'eQTL': 1,
        })

def mapping_score(path):
    if path == '':
        return 1
    else:
        return 1/len(path.split('-'))

anno_reposition['Mapping Score'] = anno_reposition['Path'].apply(mapping_score)
dti_median = drugs['interaction_group_score'].median()
anno_reposition['DTI Score Norm'] = 0
anno_reposition['DTI Score Norm'] = anno_reposition['DTI Score Norm'].where(anno_reposition['DTI Score']<=dti_median, 1)
# anno_reposition['D2V Score'] = anno_reposition['DTI Score']/anno_reposition['DTI Score'].max()*anno_reposition['RWR Prob']*anno_reposition['Direction Type Score']
# anno_reposition['D2V Score'] = anno_reposition['DTI Score Norm']*anno_reposition['Mapping Score']*anno_reposition['Direction Type Score']
anno_reposition['D2V Score'] = anno_reposition['Mapping Score']*anno_reposition['Direction Type Score']


In [33]:
drugs_rank = pd.DataFrame(columns=['Drug','CHEMBL ID','Clinical Phase','D2V Score'])
ith = 0
for drug, sub_df in anno_reposition.groupby('CHEMBL ID'):
    sub_df = sub_df.drop_duplicates('Host Gene')
    drugs_rank.loc[ith] = sub_df.iloc[0]['Drug'], drug, sub_df.iloc[0]['Clinical Phase'], sub_df['D2V Score'].sum()
    ith += 1

In [34]:
drugs_rank = drugs_rank[~drugs_rank['Drug'].isin(side_effect_map.index)]

In [35]:
# drugs_rank['Side Effect'] = drugs_rank['Drug'].map(side_effect_map).fillna('')
# drugs_rank['Side Effect Counts'] = drugs_rank['Drug'].map(side_effect_counts).fillna(0)
drugs_rank['Comorbidity'] = drugs_rank['Drug'].map(cormobidity_map).fillna('')
drugs_rank['Comorbidity Counts'] = drugs_rank['Drug'].map(cormobidity_counts).fillna(0)

In [36]:
drugs_rank['Rating Score'] = drugs_rank['D2V Score'] + drugs_rank['Comorbidity Counts']*1

In [37]:
drugs_rank = drugs_rank.sort_values('Rating Score',ascending=False)
drugs_rank.index = range(len(drugs_rank))
drugs_rank['Clinical Trial'] = 0
drugs_rank['Clinical Trial'] = drugs_rank['Clinical Trial'].where(~drugs_rank['CHEMBL ID'].isin(clinicaltrial_all), 1)

In [47]:
drugs_rank.to_csv('./drugs_rank.csv', index=False)

In [39]:
drugs_rank[drugs_rank['CHEMBL ID']=='CHEMBL163']

,Drug,CHEMBL ID,Clinical Phase,D2V Score,Comorbidity,Comorbidity Counts,Rating Score,Clinical Trial
4175,Ritonavir,CHEMBL163,4.0,1.833333,,0.0,1.833333,1


In [40]:
drugs_rank[drugs_rank['Drug']=='BOSUTINIB']

,Drug,CHEMBL ID,Clinical Phase,D2V Score,Comorbidity,Comorbidity Counts,Rating Score,Clinical Trial
2,BOSUTINIB,CHEMBL288441,4.0,29.375,"Coronary Artery Disease;Diabetes Mellitus, Type 2",2.0,31.375,0


In [41]:
cutoff = 10
print(len(drugs_rank[drugs_rank['Rating Score']>=cutoff]))
print(len(drugs_rank[drugs_rank['Clinical Trial']==1]))
print(len(drugs_rank[(drugs_rank['Rating Score']>=cutoff)&(drugs_rank['Clinical Trial']==1)]))
print(len(drugs_rank[(drugs_rank['Rating Score']>=cutoff)&(drugs_rank['Clinical Trial']==1)&(drugs_rank['Clinical Phase']==4)]))

408
189
33
24


In [42]:
cutoff = 10
drugs_rank_approv = drugs_rank[drugs_rank['Clinical Phase']==4].copy()
print(len(drugs_rank_approv[drugs_rank_approv['Rating Score']>=cutoff]))
print(len(drugs_rank_approv[drugs_rank_approv['Clinical Trial']==1]))
print(len(drugs_rank_approv[(drugs_rank_approv['Rating Score']>=cutoff)&(drugs_rank_approv['Clinical Trial']==1)]))
print(len(drugs_rank_approv[(drugs_rank_approv['Rating Score']>=cutoff)&(drugs_rank_approv['Clinical Trial']==1)&(drugs_rank_approv['Clinical Phase']==4)]))

124
134
24
24


In [43]:
len(drugs_rank['CHEMBL ID'].unique())

6023

In [44]:
len(drugs_rank[drugs_rank['Clinical Trial']==1]['CHEMBL ID'].unique())

189

In [45]:
drugs_rank_approv[(drugs_rank_approv['Rating Score']>=cutoff)&(drugs_rank_approv['Clinical Trial']==1)&(drugs_rank_approv['Clinical Phase']==4)]

,Drug,CHEMBL ID,Clinical Phase,D2V Score,Comorbidity,Comorbidity Counts,Rating Score,Clinical Trial
4,THALIDOMIDE,CHEMBL468,4.0,28.083333,,0.0,28.083333,1
6,ASPIRIN,CHEMBL25,4.0,27.833333,,0.0,27.833333,1
22,Cannabidiol,CHEMBL190461,4.0,21.833333,"Diabetes Mellitus, Type 2;Coronary Artery Disease",2.0,23.833333,1
52,LENALIDOMIDE,CHEMBL848,4.0,20.916667,,0.0,20.916667,1
89,PROGESTERONE,CHEMBL103,4.0,17.333333,,0.0,17.333333,1
94,Chlorpromazine,CHEMBL71,4.0,17.083333,,0.0,17.083333,1
95,Ketamine,CHEMBL742,4.0,17.041667,,0.0,17.041667,1
145,Ibuprofen,CHEMBL521,4.0,13.541667,"Diabetes Mellitus, Type 2",1.0,14.541667,1
147,Imatinib,CHEMBL941,4.0,14.458333,,0.0,14.458333,1
179,Atorvastatin,CHEMBL1487,4.0,13.333333,,0.0,13.333333,1


In [46]:
drugs_rank_approv[:10]

,Drug,CHEMBL ID,Clinical Phase,D2V Score,Comorbidity,Comorbidity Counts,Rating Score,Clinical Trial
2,BOSUTINIB,CHEMBL288441,4.0,29.375000,"Coronary Artery Disease;Diabetes Mellitus, Type 2",2.0,31.375000,0
4,THALIDOMIDE,CHEMBL468,4.0,28.083333,,0.0,28.083333,1
6,ASPIRIN,CHEMBL25,4.0,27.833333,,0.0,27.833333,1
12,MIDOSTAURIN,CHEMBL608533,4.0,24.750000,Essential Hypertension,1.0,25.750000,0
14,REGORAFENIB,CHEMBL1946170,4.0,24.916667,,0.0,24.916667,0
17,Pseudoephedrine,CHEMBL1590,4.0,23.708333,Essential Hypertension,1.0,24.708333,0
18,PONATINIB,CHEMBL1171837,4.0,24.583333,,0.0,24.583333,0
22,Cannabidiol,CHEMBL190461,4.0,21.833333,"Diabetes Mellitus, Type 2;Coronary Artery Disease",2.0,23.833333,1
27,ADAPALENE,CHEMBL1265,4.0,23.166667,,0.0,23.166667,0
28,PANOBINOSTAT,CHEMBL483254,4.0,23.166667,,0.0,23.166667,0


In [51]:
all_drugs = drugs['ChEMBL'].unique()

In [55]:
len(set(all_drugs) & set(clinicaltrial_all))

224

In [56]:
len(set(all_drugs) & set(clinicaltrial_approv))

159

In [91]:
drugs_rank.to_csv('./drug_candidate.csv',index=False)

---

In [26]:
anno_reposition[
    'Direction Type Score'] = anno_reposition[
        'Direction Type'].map({
            'DEG': 0.5,
            'CRISPR-Screening': 1,
            'eQTL': 1,
        })


dti_median = drugs['interaction_group_score'].median()
anno_reposition['DTI Score Norm'] = 0
anno_reposition['DTI Score Norm'] = anno_reposition['DTI Score Norm'].where(anno_reposition['DTI Score']<=dti_median, 1)
# anno_reposition['DTI Score Norm'] = drugs['interaction_group_score']/drugs['interaction_group_score'].max()


anno_reposition['SARS-CoV2'] = 1
anno_reposition['SARS-CoV2'] = anno_reposition['SARS-CoV2'].where(anno_reposition['Virus']=='SARS-CoV2', 0)

In [30]:
anno_reposition['Rating Score'] = anno_reposition['SARS-CoV2']*0.5 +\
    anno_reposition['Direction Type Score']\
    + anno_reposition['RWR Prob'] + \
    anno_reposition['DTI Score Norm'] - anno_reposition['Side Effect Counts'] + \
    anno_reposition['Comorbidity Counts']*0.1

In [31]:
anno_reposition = anno_reposition.sort_values('Rating Score', ascending=False)

In [102]:
with open('./repo_drugs.txt','w') as f:
    f.write('\n'.join(anno_reposition['Drug'].unique()))

In [104]:
drugs_chembl = pd.read_csv('./drugs_chembl.txt',sep='\t',header=None)
drugs_chembl = drugs_chembl.drop_duplicates(0)
drugs_chembl = pd.Series(index=drugs_chembl[0].values, data=drugs_chembl[1].values)
anno_reposition['ChEMBL_guess'] = anno_reposition['Drug'].map(drugs_chembl)